In [1]:
# Install necessary libraries
!pip install transformers
!pip install torch








     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 51.7 MB/s eta 0:00:00


In [2]:
# Import Pandas library
import pandas as pd
# Upload CSV file from your device
from google.colab import files
uploaded = files.upload()

# Assuming the file is named 'legal_data.csv'
file_name = list(uploaded.keys())[0]

# Read the CSV file into a Pandas DataFrame
df = pd.read_csv(file_name)

Saving filtered_dataset (1).csv to filtered_dataset (1).csv


In [3]:
 #Select a subset (20%) of the dataset
df_subset = df.sample(frac=1, random_state=42)

In [4]:
# Download NLTK resources
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
pip install networkx


In [6]:
import pandas as pd
import networkx as nx
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [7]:

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re


In [8]:
def preprocess_text(text):
    # Remove non-alphabetic characters
    text = re.sub("[^a-zA-Z]", " ", str(text))

    # Convert to lowercase
    text = text.lower()

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])

    # Stemming
    stemmer = PorterStemmer()
    text = ' '.join([stemmer.stem(word) for word in text.split()])

    return text

In [9]:
from sklearn.feature_extraction.text import CountVectorizer


In [10]:
# Preprocess the case_text column in the subset
df_subset['processed_text'] = df_subset['case_text'].apply(preprocess_text)

# Use CountVectorizer to convert text into a matrix of token counts
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df_subset['processed_text'].astype(str))  # Ensure the text is treated as a string

# Calculate cosine similarity matrix
cosine_sim_matrix = cosine_similarity(X, X)

# Convert similarity matrix to a graph
G = nx.from_numpy_array(cosine_sim_matrix)

# Run PageRank algorithm
pagerank_scores = nx.pagerank(G)

# Sort sentences based on PageRank scores
sorted_sentences = sorted(pagerank_scores.items(), key=lambda item: item[1], reverse=True)

# Calculate the number of sentences to include in the summary (top 60%)
num_sentences_in_summary = int(0.5 * len(sorted_sentences))

# Take the top 60% sentences for the summary
summary_sentences = [sentence[0] for sentence in sorted_sentences[:num_sentences_in_summary]]

# Create a new column for the summary in the subset DataFrame
df_subset['summary'] = df_subset['case_text'].apply(lambda x: ' '.join([sent for i, sent in enumerate(str(x).split('.')) if i in summary_sentences]))


In [11]:
df_subset.head()

,case_id,case_title,case_text,case_outcome,processed_text,summary
361,Case2077,Australian Broadcasting Commission v Parish (1...,The Court has power under O 15 r 8 of the Fede...,cited,court power r feder court rule order particula...,The Court has power under O 15 r 8 of the Fede...
73,Case361,P. &amp; C. Cantarella Pty Ltd v Egg Marketing...,"Moreover, I do not consider that ASIC will be ...",referred to,moreov consid asic influenc tactic forens cons...,"Moreover, I do not consider that ASIC will be ..."
374,Case2177,Capital Club Pty Ltd v Commissioner of State R...,SSAU Nominees involved a claim for a sales tax...,considered,ssau nomine involv claim sale tax exempt good ...,SSAU Nominees involved a claim for a sales tax...
155,Case824,Cadbury Schweppes Pty Ltd v Darrell Lea Chocol...,The evidence of the Cadbury experts had been e...,cited,evid cadburi expert exclud cadburi schwepp pti...,The evidence of the Cadbury experts had been e...
104,Case539,McDermott v Collien [1953] HCA 44 ; (1953) 87 ...,Parliament has evinced a general policy in s 3...,discussed,parliament evinc gener polici feder court act ...,Parliament has evinced a general policy in s 3...


In [13]:
# @title extractive_summary

from matplotlib import pyplot as plt
import seaborn as sns
df_subset.groupby('extractive_summary').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

KeyError: 'extractive_summary'

In [16]:
# Calculate the length of each 'case_text' and 'bart_summaries'
df_subset['summary_length'] = df_subset['summary'].str.len()
df_subset['casetext_length'] = df_subset['case_text'].str.len()

# Compute the difference in length between 'case_text' and 'bart_summaries'
df_subset['length_difference'] = df_subset['casetext_length'] - df_subset['summary_length']

# Calculate the average length difference
average_length_difference = df_subset['length_difference'].mean()

print("Average length difference between 'summary' and 'casetext':", average_length_difference)

Average length difference between 'summary' and 'casetext': 228.834


In [17]:
df_subset.head()

,case_id,case_title,case_text,case_outcome,processed_text,summary,summary_length,casetext_length,length_difference
361,Case2077,Australian Broadcasting Commission v Parish (1...,The Court has power under O 15 r 8 of the Fede...,cited,court power r feder court rule order particula...,The Court has power under O 15 r 8 of the Fede...,382,506,124
73,Case361,P. &amp; C. Cantarella Pty Ltd v Egg Marketing...,"Moreover, I do not consider that ASIC will be ...",referred to,moreov consid asic influenc tactic forens cons...,"Moreover, I do not consider that ASIC will be ...",603,919,316
374,Case2177,Capital Club Pty Ltd v Commissioner of State R...,SSAU Nominees involved a claim for a sales tax...,considered,ssau nomine involv claim sale tax exempt good ...,SSAU Nominees involved a claim for a sales tax...,785,861,76
155,Case824,Cadbury Schweppes Pty Ltd v Darrell Lea Chocol...,The evidence of the Cadbury experts had been e...,cited,evid cadburi expert exclud cadburi schwepp pti...,The evidence of the Cadbury experts had been e...,839,840,1
104,Case539,McDermott v Collien [1953] HCA 44 ; (1953) 87 ...,Parliament has evinced a general policy in s 3...,discussed,parliament evinc gener polici feder court act ...,Parliament has evinced a general policy in s 3...,728,829,101


In [18]:
import pandas as pd

# Assuming 'df' is your DataFrame with the 'extractive_summary' column
max_summary_length = 0
min_summary_length = float('inf')  # Initialize with a large value

# Iterate over each summary in the 'extractive_summary' column
for summary in df_subset['summary']:
    summary_length = len(summary.split())  # Calculate the length of the summary
    if summary_length > max_summary_length:
        max_summary_length = summary_length  # Update the maximum length if needed
    if summary_length < min_summary_length:
        min_summary_length = summary_length  # Update the minimum length if needed

print("Maximum length of extractive summaries:", max_summary_length)
print("Minimum length of extractive summaries:", min_summary_length)


Maximum length of extractive summaries: 165
Minimum length of extractive summaries: 12


In [19]:
total_rows = df.shape[0]
print("Total number of rows in the dataset:", total_rows)


Total number of rows in the dataset: 500


In [20]:
import pandas as pd

from transformers import T5ForConditionalGeneration, T5Tokenizer
from nltk.translate.bleu_score import corpus_bleu



# Load pre-trained T5 model and tokenizer for summarization
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [21]:

# Create a new column for extractive summaries
df_subset['abstractive_summary'] = ''

# Loop through the DataFrame and generate extractive summaries
for index, row in df_subset.iterrows():
    case_text = str(row['summary'])

    # Tokenize and encode the input text
    input_ids = tokenizer.encode( case_text, return_tensors="pt", max_length=512, truncation=True)

    # Generate extractive summary
    summary_ids = model.generate(input_ids, max_length=150, length_penalty=2.0, num_beams=4)
    abstractive_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    # Add the generated summary to the 'extractive_summary' column
    df_subset.at[index, 'abstractive_summary'] = abstractive_summary

In [22]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import pandas as pd

# Assuming 'df' is your DataFrame containing the generated extractive summaries and reference summaries
# Replace 'generated_summary' and 'reference_summary' with the actual column names if they are different
generated_summaries = df_subset['abstractive_summary'].astype(str).tolist()
reference_summaries = df_subset['summary'].astype(str).tolist()

# Initialize lists to store BLEU scores
bleu1_scores = []
bleu2_scores = []
bleu3_scores = []

# Compute BLEU scores for each pair of generated summary and reference summary
for generated_summary, reference_summary in zip(generated_summaries, reference_summaries):
    # Tokenize the summaries into lists of words
    generated_tokens = generated_summary.split()
    reference_tokens = reference_summary.split()

    # Compute BLEU-1, BLEU-2, and BLEU-3 scores
    bleu1 = sentence_bleu([reference_tokens], generated_tokens, weights=(1, 0, 0), smoothing_function=SmoothingFunction().method1)
    bleu2 = sentence_bleu([reference_tokens], generated_tokens, weights=(0.5, 0.5, 0), smoothing_function=SmoothingFunction().method1)
    bleu3 = sentence_bleu([reference_tokens], generated_tokens, weights=(0.33, 0.33, 0.33), smoothing_function=SmoothingFunction().method1)

    # Append the scores to the respective lists
    bleu1_scores.append(bleu1)
    bleu2_scores.append(bleu2)
    bleu3_scores.append(bleu3)

# Add BLEU scores as new columns in the DataFrame
df_subset['bleu1_score'] = bleu1_scores
df_subset['bleu2_score'] = bleu2_scores
df_subset['bleu3_score'] = bleu3_scores

# Optionally, you can also print the average BLEU scores
print("Average BLEU-1 Score:", pd.Series(bleu1_scores).mean())
print("Average BLEU-2 Score:", pd.Series(bleu2_scores).mean())
print("Average BLEU-3 Score:", pd.Series(bleu3_scores).mean())


Average BLEU-1 Score: 0.5727822639410548
Average BLEU-2 Score: 0.5593117812304614
Average BLEU-3 Score: 0.5512806959052746


In [23]:
!pip install rouge


In [24]:
import pandas as pd
from rouge import Rouge

# Assuming 'df_subset' is your DataFrame containing the abstractive summaries and reference summaries
# Replace 'generated_summary' and 'reference_summary' with the actual column names if they are different
generated_summaries = df_subset['abstractive_summary'].astype(str).tolist()
reference_summaries = df_subset['summary'].astype(str).tolist()

# Initialize the Rouge object
rouge = Rouge()

# Initialize lists to store ROUGE scores
rouge1_scores = []
rouge2_scores = []
rouge3_scores = []

# Compute ROUGE scores for each pair of generated summary and reference summary
for generated_summary, reference_summary in zip(generated_summaries, reference_summaries):
    # Compute ROUGE scores
    scores = rouge.get_scores(generated_summary, reference_summary)[0]

    # Append ROUGE-1 and ROUGE-2 scores
    rouge1_scores.append(scores['rouge-1']['f'])
    rouge2_scores.append(scores['rouge-2']['f'])

    # Check if the 'rouge-3' key exists in the scores dictionary
    if 'rouge-3' in scores:
        rouge3_scores.append(scores['rouge-3']['f'])
    else:
        rouge3_scores.append(scores['rouge-2']['f'])  # Use ROUGE-2 score as a fallback if ROUGE-3 is not available

# Add ROUGE scores as new columns in the DataFrame
df_subset['rouge1_score'] = rouge1_scores
df_subset['rouge2_score'] = rouge2_scores
df_subset['rouge3_score'] = rouge3_scores

# Optionally, you can also print the average ROUGE scores
print("Average ROUGE-1 Score:", pd.Series(rouge1_scores).mean())
print("Average ROUGE-2 Score:", pd.Series(rouge2_scores).mean())
print("Average ROUGE-3 Score:", pd.Series(rouge3_scores).mean())


Average ROUGE-1 Score: 0.7644860393425401
Average ROUGE-2 Score: 0.7210365737450254
Average ROUGE-3 Score: 0.7210365737450254


In [25]:
# Calculate the length of each 'case_text' and 'bart_summaries'
df_subset['summary_length'] = df_subset['summary'].str.len()
df_subset['abs_summaries_length'] = df_subset['abstractive_summary'].str.len()

# Compute the difference in length between 'case_text' and 'bart_summaries'
df_subset['length_difference'] = df_subset['summary_length'] - df_subset['abs_summaries_length']

# Calculate the average length difference
average_length_difference = df_subset['length_difference'].mean()

print("Average length difference between 'summary' and 'abstractive_summaries':", average_length_difference)

Average length difference between 'summary' and 'abstractive_summaries': 126.762


In [26]:
# Calculate the length of each 'case_text' and 'bart_summaries'
df_subset['summary_length'] = df_subset['summary'].str.len()
df_subset['casetext_length'] = df_subset['case_text'].str.len()

# Compute the difference in length between 'case_text' and 'bart_summaries'
df_subset['length_difference'] = df_subset['casetext_length'] - df_subset['summary_length']

# Calculate the average length difference
average_length_difference = df_subset['length_difference'].mean()

print("Average length difference between 'summary' and 'casetext':", average_length_difference)

Average length difference between 'summary' and 'casetext': 228.834


In [27]:
print(df_subset)

      case_id                                         case_title  \
361  Case2077  Australian Broadcasting Commission v Parish (1...   
73    Case361  P. &amp; C. Cantarella Pty Ltd v Egg Marketing...   
374  Case2177  Capital Club Pty Ltd v Commissioner of State R...   
155   Case824  Cadbury Schweppes Pty Ltd v Darrell Lea Chocol...   
104   Case539  McDermott v Collien [1953] HCA 44 ; (1953) 87 ...   
..        ...                                                ...   
106   Case546  Paras v Public Service Body Head of the Depart...   
270  Case1525  Caterpillar Inc v John Deere Limited and Other...   
348  Case2018  Xat Ky v Australvic Property Management Pty Lt...   
435  Case2568  Spencer Constructions Pty Ltd v G&amp;M Aldrid...   
102   Case537  Lang v Australian Consolidated Press Limited [...   

                                             case_text case_outcome  \
361  The Court has power under O 15 r 8 of the Fede...        cited   
73   Moreover, I do not consider that ASI